In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from tqdm import tqdm


In [37]:
class Distribution(nn.Module):
    def __init__(self, size=1000):
        super().__init__()
        self.mu = nn.Parameter(torch.tensor(0.0), requires_grad=True)
        self.std = nn.Parameter(torch.tensor(1.0), requires_grad=True)
        self.size = size
    def forward(self):
        return self.mu + self.std * torch.normal(0, 1, size=(1, self.size)).flatten()

class LinearRegression(nn.Module):
    def __init__(self, size=1000):
        super().__init__()
        self.m = Distribution(size=size)
        self.b = Distribution(size=size)
    def forward(self, x, y):
        m = self.m()
        b = self.b()
        x = torch.tensor(x)
        y = torch.tensor(y)
        
        total_loss = (((y - (torch.einsum('m,x -> xm', m, x) + b).transpose(0, 1)) ** 2).sum(axis=0) * (1 / len(m))).sum() * (1 / len(x))
        
        print('TOTAL LOSS:', total_loss)
        
        # Minimize loss by maximizing this value
        return total_loss

In [20]:
def training(x, y, epochs=5):
    model = LinearRegression()
    optimizer = optim.Adam(model.parameters())
    
    for epoch in tqdm(range(epochs), desc="Training..."):
        # Zeros gradiant for training
        optimizer.zero_grad()
        
        # Calculates likelihood
        loglik = model(x, y)
        e = torch.mean(loglik)
        
        # Updates parameters
        e.backward()
        optimizer.step()
        
    return model

In [21]:
df = pd.read_csv('data/Advertising.csv').drop('Unnamed: 0', axis=1)
df

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [22]:
x = df.TV.to_numpy()
y = df.Sales.to_numpy()

In [38]:
res = training(x, y, epochs=10)
res

Training...: 100%|██████████| 10/10 [00:00<00:00, 66.82it/s]

TOTAL LOSS: tensor(28794.9444, dtype=torch.float64, grad_fn=<MulBackward0>)
TOTAL LOSS: tensor(28695.0260, dtype=torch.float64, grad_fn=<MulBackward0>)
TOTAL LOSS: tensor(29376.0443, dtype=torch.float64, grad_fn=<MulBackward0>)
TOTAL LOSS: tensor(29913.7789, dtype=torch.float64, grad_fn=<MulBackward0>)
TOTAL LOSS: tensor(29437.1834, dtype=torch.float64, grad_fn=<MulBackward0>)
TOTAL LOSS: tensor(28441.4475, dtype=torch.float64, grad_fn=<MulBackward0>)
TOTAL LOSS: tensor(28787.5172, dtype=torch.float64, grad_fn=<MulBackward0>)
TOTAL LOSS: tensor(30509.7244, dtype=torch.float64, grad_fn=<MulBackward0>)
TOTAL LOSS: tensor(28388.8424, dtype=torch.float64, grad_fn=<MulBackward0>)
TOTAL LOSS: tensor(29140.7681, dtype=torch.float64, grad_fn=<MulBackward0>)


LinearRegression(
  (m): Distribution()
  (b): Distribution()
)

In [39]:
print(res.m.mu)

Parameter containing:
tensor(0.0094, requires_grad=True)


In [40]:
print(res.m.std)

Parameter containing:
tensor(0.9900, requires_grad=True)


In [41]:
print(res.b.mu)

Parameter containing:
tensor(0.0095, requires_grad=True)


In [42]:
print(res.b.std)

Parameter containing:
tensor(1.0029, requires_grad=True)
